In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import dizionari
import numpy as np

template = 'seaborn'

data = pd.read_csv("../data/ISTAT_Italian_Excursion_2014_2022.csv")

# CREO UN NUOVO DATAFRAME PIU' VERSATILE

# RIMOZIONE ATTRIBUTI SUPERFLUI
colonne_da_eliminare = ["ID", "progind", "staciv4", "ateco3", "ORARIO", "RAPP", "CODPROF7", "progesc", "PARTESC", "NPARTESC"]
df = data.drop(columns=colonne_da_eliminare)

# RINOMINA DEGLI ATTRIBUTI
df.rename(columns = dizionari.dizionario_nomi, inplace=True)

# ADATTAMENTO COLONNE
df['Data'] = pd.to_datetime(df['Anno'].apply(str) + '-' + df['Mese'].apply(str), format='%Y-%m')
df["Data"] = df["Data"].dt.strftime("%Y-%m")

# MAPPING
df['Età'] = df['Età'].map(dizionari.eta_mapping)
df['Regione'] = (df['Regione']/10).astype(int)
df['Zona d Italia'] = df['Zona d Italia'].map(dizionari.zona_mapping)
df['Livello di Istruzione'] = df['Livello di Istruzione'].map(dizionari.livello_di_istruzione_mapping)
df['Italia/Estero'] = df['Italia/Estero'].map(dizionari.destinazione_italia_estero_mapping)
df['Motivo dell escursione'] = df['Motivo dell escursione'].map(dizionari.motivo_escursione_mapping)
df['Sesso'] = df['Sesso'].map(dizionari.sesso_mapping)
df['Stato di impiego'] = df['Stato di impiego'].map(dizionari.stato_di_impiego_mapping)
df['Mezzo'] = df['Mezzo'].map(dizionari.mezzo_mapping)
df['Grado di impiego'] = df['Grado di impiego'].map(dizionari.grado_di_impiego_mapping)




In [9]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MATERIA])
app.title = 'Escursioni in Italia'

linchart_dropdown = dcc.Dropdown(
    id="radio-linecharts-id",
    options=[
        {"label": "Età", "value": "Età"},
        {"label": "Estero", "value": "Destinazione (regioni italiane o paesi esteri)"},
    ],
    value="Età",
)

map_anno_slider = dcc.Slider(
    2014, 2022, 1,
    value = 2020,
    id='map-anno-id',
    marks={anno: str(anno) for anno in range(2014, 2023)}
)

map_stagioni_slider = dcc.Slider(
    0, 4, 1,
    value = 2,
    id='map-stagioni-id',
    marks={
        0: 'Anno intero',
        1: 'Gennaio-Marzo',
        2: 'Aprile-Giugno',
        3: 'Luglio-Settembre',
        4: 'Ottobre-Dicembre'
        }
)

sunburst_dropdown =dcc.Dropdown(
    id='radio-sunburst-id',
    options=[{"label": "Zona d Italia", "value": "Zona d Italia"},
        {"label": "Livello di Istruzione", "value": "Livello di Istruzione"},
        ],
    value='Zona d Italia',
)



hist_feature_dropdown = dcc.Dropdown(
    id='hist-feature',
    options=[{"label": "Mezzo", "value": "Mezzo"},
        {"label": "Motivo dell escursione", "value": "Motivo dell escursione"},
        {"label": "Grado di impiego", "value": "Grado di impiego"}
        ],
    value='Motivo dell escursione'
)

# Layout dell'app
app.layout = dbc.Container([
    html.H1("Analisi Escursioni in Italia",
            style={'textAlign': 'center',
                   'padding': '5px'}),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              map_anno_slider,
              map_stagioni_slider,
            ])
        , width=6),
    ], justify='center'),

    dbc.Row([
        dbc.Col(
            [
                html.Label("Regioni di partenza", style={'text-align': 'center', 'font-weight': 'bold'}),
                dcc.Graph(id='map_partenze', style={'height': '100%', 'width': '100%'})
            ],
            width=5,
            style={'height': '100%', 'padding': '0'}),
        dbc.Col(
            [
                html.Label("Province di arrivo", style={'text-align': 'center', 'font-weight': 'bold'}),
                dcc.Graph(id='map_arrivi', style={'height': '100%', 'width': '100%'})
            ],
            width=5,
            style={'height': '100%', 'padding': '0'}),
    ], style={'height': '35vh', 'justify-content':'center'}),

    html.Br(),
    html.Br(),

    dbc.Row([
        dbc.Col(
            dbc.Stack([
              linchart_dropdown,
              ])
            , width=4),

        dbc.Col([
            dbc.Stack([
              hist_feature_dropdown,
            ])
        ], width=4, align="center"),

        dbc.Col([
            dbc.Stack([
                sunburst_dropdown,
            ])
        ], width=4),
    ]),
    dbc.Row([
        dbc.Col(
            dbc.Stack([
              dcc.Graph(id='graph')
              ])
            , width=4),

        
        dbc.Col([
            dbc.Stack([
              dcc.Graph(id='histogram')
            ])
        ], width=4),

        dbc.Col([
            dbc.Stack([
                dcc.Graph(id='sunburst-graph')
            ])
        ], width=4),
    ]),
    
], fluid=True)

# LINECHART - Evoluzione degli spostamenti negli anni (suddivisi per: [Età, Stati Esteri])
@app.callback(
    Output('graph', 'figure'),
    [Input('radio-linecharts-id', 'value')]
)
def update_line_chart(linchart_dropdown):
    conteggio = df.groupby(['Anno', linchart_dropdown]).size().reset_index(name='Numero escursioni')
    
    colori = ['#FF69B4', '#FFA500',  '#2E8B57','#6A5ACD']

    if linchart_dropdown == "Destinazione (regioni italiane o paesi esteri)":
        linchart_dropdown = "Stati Esteri"
        conteggio.rename(columns={"Destinazione (regioni italiane o paesi esteri)" : linchart_dropdown}, inplace=True)
        conteggio = conteggio[conteggio[linchart_dropdown] > 100]
        conteggio['Stati Esteri'] = conteggio[linchart_dropdown].map(dizionari.destinazione_regioni_stato_mapping)
        
        valori_da_visualizzare = ['Austria', 'Francia', 'Germania', 'Slovenia', 'Croazia', 'Paesi Extra-UE']
        conteggio = conteggio[conteggio['Stati Esteri'].isin(valori_da_visualizzare)]
        
        colori = [
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6', '#A633FF', '#33FFA6',
            '#FF8C33', '#8CFF33', '#338CFF', '#FF338C', '#8C33FF', '#33FF8C',
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6', '#A633FF', '#33FFA6',
            '#FF8C33', '#8CFF33', '#338CFF', '#FF338C', '#8C33FF', '#33FF8C',
            '#FF5733', '#33FF57', '#3357FF', '#FF33A6'
        ]

    fig = px.line(conteggio,
                  x='Anno',
                  y='Numero escursioni',
                  color='Stati Esteri' if linchart_dropdown == "Stati Esteri" else linchart_dropdown,
                  title=f'Evoluzione degli spostamenti negli anni rispetto a [{linchart_dropdown}]',
                  color_discrete_sequence=colori,
                  template=template,
                  line_shape='spline')

    fig.update_layout(
        xaxis_title = '',
        margin=dict(l=50, r=50, t=50, b=50))
    
    return fig

# CHOROPLET MAPS - Partenze e arrivi
@app.callback(
    Output('map_partenze', 'figure'),
    Output('map_arrivi', 'figure'),
    [Input('map-anno-id', 'value'),
     Input('map-stagioni-id', 'value')]
)
def update_map(map_anno, map_stagioni):
    maschera_anno = df["Anno"] == map_anno
    
    if map_stagioni == 0: # 0 : 'Anno intero'
        maschera_stagioni = True
    else:
        maschera_stagioni = df['Stagione'] == map_stagioni

    conteggio_partenze = df[maschera_anno & maschera_stagioni]['Regione'].value_counts().reset_index(name="N. Partenze")
    
    tutte_regioni = pd.DataFrame({'Regione': list(range(1, 21))})
    conteggio_partenze = tutte_regioni.merge(conteggio_partenze, on='Regione', how='left')
    conteggio_partenze['N. Partenze'] = conteggio_partenze['N. Partenze'].fillna(0)
    conteggio_partenze['Reg'] = conteggio_partenze['Regione'].map(dizionari.destinazione_regioni_stato_mapping)

    fig1 = px.choropleth(
        conteggio_partenze,
        geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_regions.geojson',
        locations='Regione',
        color='N. Partenze',
        color_continuous_scale=[[0.0, 'lightgray'],
                                [0.01, 'lightgray'],
                                [0.01, 'azure'],
                                [1, 'royalblue']],
        featureidkey='properties.reg_istat_code_num',
        range_color=(1, max(conteggio_partenze['N. Partenze'])),
        hover_data={'Regione' : False, 'Reg' : True})
    
    fig1.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig1.update_geos(fitbounds="locations", visible=True)

    conteggio_arrivi = df[maschera_anno & maschera_stagioni]['Destinazione (città)'].value_counts().reset_index(name="N. Arrivi")

    tutte_regioni = pd.DataFrame({'Destinazione (città)': list(range(1, 112))})
    conteggio_arrivi = tutte_regioni.merge(conteggio_arrivi, on='Destinazione (città)', how='left')
    conteggio_arrivi['N. Arrivi'] = conteggio_arrivi['N. Arrivi'].fillna(0)
    conteggio_arrivi['Città'] = conteggio_arrivi['Destinazione (città)'].map(dizionari.destinazione_citta_mapping)

    fig2 = px.choropleth(
        conteggio_arrivi,
        geojson='https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson',
        locations='Destinazione (città)',
        color='N. Arrivi',
        color_continuous_scale=[[0.0, 'lightgray'],
                                [0.01, 'lightgray'],
                                [0.01, 'mistyrose'],
                                [1, 'darkred']],
        featureidkey='properties.prov_istat_code_num',
        range_color=(0, max(conteggio_arrivi['N. Arrivi'])),
        hover_data={'Destinazione (città)': False, 'Città': True})

    fig2.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig2.update_geos(fitbounds="locations", visible=True)

    return fig1,fig2


color_map = {
    'Nord': '#1f77b4',  # blue
    'Centro': '#ff7f0e',  # orange
    'Sud': '#2ca02c'  # green
}


# SUNBURST - Zone d'Italia e Livello di istruzione
@app.callback(
    Output('sunburst-graph', 'figure'),
    Input('radio-sunburst-id', 'value')
)
def update_sunburst(sunburst_dropdown):
    if sunburst_dropdown == "Zona d Italia":
        category_list = ['Zona d Italia', 'Livello di Istruzione', 'Italia/Estero']
    elif sunburst_dropdown == "Livello di Istruzione":
        category_list = ['Livello di Istruzione', "Zona d Italia", 'Italia/Estero']
    
    fig_sunburst = px.sunburst(df, path=category_list, template=template, color_discrete_map=color_map)
    
    fig_sunburst.update_layout(margin=dict(l=50, r=50, t=50, b=50))
    
    return fig_sunburst

# HISTOGRAM - Impiego e Mezzo utilizzato
@app.callback(
    Output('histogram', 'figure'),
    [Input('hist-feature', 'value')]
)
def update_histogram(selected_feature):
  
    fig = px.histogram(
        df,
        x=selected_feature,
        color='Zona d Italia',
        opacity=0.7,
        title=f'Istogramma in base a [{selected_feature}]',
        labels=selected_feature,
        color_discrete_map=color_map,
        template=template,
        category_orders= {'Zona d Italia': ['Nord', 'Centro', 'Sud']}
    ).update_xaxes(categoryorder='total descending')
    
    fig.update_layout(
        bargap=0.01,
        bargroupgap=0.05,
        xaxis_title="",
        yaxis_title='Num. Escursioni', 
        showlegend=True,
        margin=dict(l=0, r=0, t=50, b=30))

    return fig

# Esecuzione dell'app
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="external", port = "8051")


# Walter Signoretti
# Andrea Cattarinich

Dash app running on http://127.0.0.1:8051/
